In [1]:
# Importing the necessary libraries #
import torch
import gc
import numpy as np
import torch.nn as nn
import torchvision.transforms.functional as F
import lightning as L
from typing import List
from lightning.pytorch import Trainer
from torchvision.datasets import MNIST
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, Subset
from sklearn.metrics import precision_score
from torchmetrics import Accuracy
from sklearn.model_selection import train_test_split
import os

In [2]:
torch.cuda.empty_cache()
torch.set_float32_matmul_precision("medium")

In [3]:
import wandb
from lightning.pytorch.loggers import WandbLogger
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ae21b105 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
# Function to give the activation function #
def create_activation_function(activation : str = "ReLU"):
    possible_activations = ["ReLU", "Mish", "GELU", "SELU", "SiLU", "LeakyReLU" ]
    # Assertion to be made for the activations possible #
    assert activation in possible_activations, f"activation not in {possible_activations}"

    if activation == "ReLU":
        return nn.ReLU()
    elif activation == "GELU":
        return nn.GELU()
    elif activation == "SiLU":
        return nn.SiLU()
    elif activation == "SELU":
        return nn.SELU()
    elif activation == "Mish":
        return nn.Mish()
    else:
        return nn.LeakyReLU()

In [5]:
class CNN_(nn.Module):
    def __init__(self, config = None):
        super().__init__()
        # Configuration to build the CNN #
        self.config = config
        
        # Some assertions to be made #
        assert config["no_of_conv_blocks"]==len(config["no_of_filters"]), "The filter number do not match with number of conv layers"
        assert config["no_of_conv_blocks"]==len(config["filter_sizes"]), "The filter sizes do not match with number of conv layers"
        assert config["no_of_conv_blocks"]==len(config["conv_strides"]), "The strides do not match with number of conv layers"
        assert config["no_of_conv_blocks"]==len(config["conv_padding"]), "The padding do not match with number of conv layers"
        assert config["no_of_conv_blocks"]==len(config["max_pooling_stride"]), "The max pooling stride do not match with number of conv layers"

        # building the convolution blocks #
        conv_blocks = []
        for block_no in range(config["no_of_conv_blocks"]):
            # Getting the hyper-parameters from the config #
            if block_no == 0:
                in_channels = config["input_channels"]
            else:
                in_channels = config["no_of_filters"][block_no-1]
            out_channels = config["no_of_filters"][block_no]
            filter_size = config["filter_sizes"][block_no]
            stride = config["conv_strides"][block_no]
            padding = config["conv_padding"][block_no]
            if padding == None:
                padding = int((filter_size - 1)/2) if filter_size > 1 else 0
            # Defining the block to add to conv_blocks #
            block_add = nn.Sequential(
                nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=filter_size, stride=stride, padding=padding),
                nn.BatchNorm2d(num_features=out_channels) if config["batch_norm_conv"] else nn.Identity(),
                create_activation_function(activation=config["conv_activation"][block_no]),
                nn.MaxPool2d(kernel_size=config["max_pooling_kernel_size"][block_no],stride=config["max_pooling_stride"][block_no]) if config["max_pooling_stride"][block_no] != None else nn.Identity(),
                nn.Dropout(config["dropout_conv"]) if config["dropout_conv"]>0 else nn.Identity(),
            )
            # Appending the blocks to the total #
            conv_blocks.append(block_add)

        # Converting the list to a sequential module #
        self.conv_blocks = nn.Sequential(*conv_blocks)

        # Calculating the size of the output #
        dummy_in = torch.randn(size=(1, config["input_channels"],config["input_size"][0], config["input_size"][1]))
        dummy_out = self.conv_blocks(dummy_in).flatten()
        flat_size = len(dummy_out)

        # building the fc blocks #
        fc_blocks = []
        for block_no in range(config["no_of_fc_layers"]):
            if block_no == 0:
                in_channels = flat_size
            else:
                in_channels = config["fc_neurons"][block_no-1]
            out_channels = config["fc_neurons"][block_no]
            block_add = nn.Sequential(
                nn.Linear(in_features=in_channels, out_features=out_channels),
                nn.BatchNorm1d(out_channels) if config["batch_norm_fc"] else nn.Identity(),
                create_activation_function(activation=config["fc_activations"][block_no]),
                nn.Dropout(config["dropout_fc"]) if config["dropout_fc"]>0 else nn.Identity(),
            )
            # Appending to the fc final
            fc_blocks.append(block_add)

        # converting the list to a sequential module #
        self.fc_layers = nn.Sequential(*fc_blocks)

        # Output layer #
        self.output_layer = nn.Sequential(
            nn.Linear(in_features=config["fc_neurons"][-1], out_features=config["num_classes"]),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv_blocks(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        x = self.output_layer(x)
        return x

In [6]:
class Lightning_CNN(L.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.save_hyperparameters()

        # Define the model
        self.model = CNN_(config=config)

        # Defining the loss and optimizers
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr = config["learning_rate"])

        # Defining the metrics
        self.acc_metric = Accuracy(task="multiclass", num_classes=config["num_classes"], average="weighted")

    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        input_, target_ = batch
        output_ = self(input_)
        # Finding the loss to backprop #
        loss = self.loss_fn(output_, target_)
        # Logging the metrics #
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        input_, target_ = batch
        output_ = self(input_)
        # Finding the loss to backprop #
        loss = self.loss_fn(output_, target_)

        output_pred = torch.argmax(output_, dim=1) 
        acc = self.acc_metric(output_pred, target_)
        # Logging the metrics #
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_acc", acc, on_step=False, on_epoch=True,prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        input_, target_ = batch
        output_ = self(input_)
        # Finding the loss to backprop #
        loss = self.loss_fn(output_, target_)
        
        output_pred = torch.argmax(output_, dim=1) 
        acc = self.acc_metric(output_pred, target_)
        # Logging the metrics #
        self.log("test_loss", loss, prog_bar=True, logger=True, sync_dist=True)
        self.log("test_acc", acc, prog_bar=True, logger=True, sync_dist=True)
        return loss
    
    def configure_optimizers(self):
        return self.optimizer


In [7]:
# class to orient (all images to landscape)
class OrientReshape:
    def __init__(self, size = (256, 256)):
        self.size = size
    
    def __call__(self, img):
        # rotate the image to landscape if potrait #
        if img.height > img.width:
            img = img.rotate(90, expand = True)
        # Reshape to target dimension #
        img = F.resize(img, size = self.size)

        return img
    

# Data augementation and transforms
def create_data_augment_compose(input_size = (256, 256)):
    data_transforms = {
        "orient_" : transforms.Compose([
            OrientReshape(size=input_size),
            transforms.ToTensor()
        ]),
        "train_" : transforms.Compose([
            transforms.RandomHorizontalFlip(p = 0.3),
            transforms.RandomVerticalFlip(p = 0.3),
            transforms.RandomRotation(degrees=15),
            transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.3),
            transforms.GaussianBlur(kernel_size=3),
            transforms.ToTensor(),
            transforms.RandomErasing(p = 0.2, scale=(0.02, 0.075)),
        ])
    }

    return data_transforms

# Create a dataset with the image folders
def create_dataset_image_folder(path_, input_size = (256,256)):
    # Getting the transform
    data_transforms = create_data_augment_compose(input_size)
    # Path to dataset
    data_dir = path_ #os.path.join(os.path.abspath(""), "nature_12K/inaturalist_12K/train/") 
    # Creating dataset
    full_dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms["orient_"])
    # Getting the labels for stratified split
    labels = [sample[1] for sample in full_dataset.samples]

    # Stratified split
    train_indices, val_indices = train_test_split(np.arange(len(labels)), test_size=0.2, stratify=labels, random_state=42)

    # Create subsets
    train_dataset = Subset(full_dataset, train_indices)
    val_dataset = Subset(full_dataset, val_indices)
    
    return train_dataset, val_dataset, data_transforms

def create_dataloaders(batch_size, num_workers, train_dataset, val_dataset, is_data_aug, data_transforms):
    # Transforming the dataset with transforms
    if is_data_aug:
        train_dataset.dataset.transform = data_transforms['train_']
    else:
        train_dataset.dataset.transform = data_transforms['orient_']

    val_dataset.dataset.transform = data_transforms['orient_']

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers,drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_loader, val_loader

In [8]:
def trainloop_sweep(config = None):
    torch.cuda.empty_cache()
    run = wandb.init(config=config)
    config = dict(wandb.config)
    run_name = f"bs_{config["batch_size"]}_layer_act_{config["conv_activation"]}_epoch_{config["epochs"]}_filter_mul_{config["filter_mul"]}_num_filters_{config["no_of_filters"]}"
    run.name = run_name

    # Closing the runs which are not possible
    if config["filter_mul"] == 2 and config["no_of_filters"]>=32:
        print("Not a good combination !!!")
        wandb.finish()
        return
    
    if config["filter_mul"] == 1 and config["no_of_filters"]==512:
        print("Not a good combination !!!")
        wandb.finish()
        return
    
    if config["filter_mul"] == 0.5 and config["no_of_filters"]<=64:
        print("Not a good combination !!!")
        wandb.finish()
        return
    
    # rechanging the configs to match the original config
    config["conv_activation"] = [config["conv_activation"]]*5
    nf, fm = config["no_of_filters"], config["filter_mul"]
    config["no_of_filters"] = list(map(int,[nf, nf*fm, nf*(fm**2), nf*(fm**3), nf*(fm**4)]))
    config["filter_sizes"] = [config["conv_1_filter_size"], config["conv_2_filter_size"], config["conv_3_filter_size"], config["conv_4_filter_size"], config["conv_5_filter_size"]]
    config["max_pooling_kernel_size"] = [config["mp_layer1_kernel_size"], config["mp_layer2_kernel_size"], config["mp_layer3_kernel_size"], config["mp_layer4_kernel_size"], config["mp_layer5_kernel_size"]]
    config["max_pooling_stride"] = [config["mp_layer1_stride"], config["mp_layer2_stride"], config["mp_layer3_stride"], config["mp_layer4_stride"], config["mp_layer5_stride"]]
    config["fc_activations"] = [config["fc_activations"]]
    config["fc_neurons"] = [config["fc_neurons"]]
    config["num_workers"] = 2
        
    try:
        # Loading the dataloaders
        train_dataset, val_dataset, data_transforms = create_dataset_image_folder(path_=os.path.join(os.path.abspath(""), "nature_12K/inaturalist_12K/train/"), input_size=config["input_size"])
        train_loader, val_loader = create_dataloaders(batch_size=config["batch_size"], num_workers=config["num_workers"], train_dataset=train_dataset, val_dataset=val_dataset, is_data_aug=config["data_aug"], data_transforms = data_transforms)

        wandb_logger = WandbLogger(log_model=False)
        model = Lightning_CNN(config=config)
        trainer = Trainer(max_epochs=config["epochs"], accelerator="auto", logger=wandb_logger)
        trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
        wandb.finish()

    except RuntimeError as e:
        if "out of memory" in str(e):
            print("⚠️ CUDA Out Of Memory — skipping this run.")
            gc.collect()
            torch.cuda.empty_cache()
        else:
            raise e

    finally:
      wandb.finish()
      del model

In [9]:
sweep_config = {
    "name" : "Q1_CNN_",
    "method" : "random",
    "metric" : {
        "name" : "val_acc",
        "goal" : "maximize"
    },
    "parameters" : {
        "learning_rate" : {"values" : [0.001, 0.0005]},
        "no_of_conv_blocks" : {"values" : [5]},
        "input_size" : {"values" : [(256, 256)]},
        "input_channels" : {"values" : [3]},
        "num_classes" : {"values" : [10]},
        "no_of_filters" : {"values" : [8, 16, 32, 64, 128, 256, 512]},
        "filter_mul" : {"values" : [0.5, 1, 2]},
        "conv_activation" : {"values" : ["ReLU", "GELU", "SiLU", "SELU", "Mish", "LeakyReLU"]},
        # Layer wise kernel size
        "conv_1_filter_size" : {"values" : [3, 5, 7]},
        "conv_2_filter_size" : {"values" : [3, 5, 7]},
        "conv_3_filter_size" : {"values" : [3, 5, 7]},
        "conv_4_filter_size" : {"values" : [3, 5, 7]},
        "conv_5_filter_size" : {"values" : [3, 5, 7]},
        # Strides are kept 1
        "conv_strides" : {"values" : [(1, 1, 1, 1, 1)] },
        "conv_padding" : {"values" : [(None, None, None, None, None)]}, # Use None if you want no reduction in size of image (stride = 1)
        # Layer wise max pooling params
        "mp_layer1_kernel_size" : {"values" : [2, 3, 5]},
        "mp_layer1_stride" : {"values" : [2, 3]},
        "mp_layer2_kernel_size" : {"values" : [2, 3, 5]},
        "mp_layer2_stride" : {"values" : [2, 3]},
        "mp_layer3_kernel_size" : {"values" : [2, 3, 5]},
        "mp_layer3_stride" : {"values" : [2, 3]},
        "mp_layer4_kernel_size" : {"values" : [2, 3]},
        "mp_layer4_stride" : {"values" : [2, 3]},
        "mp_layer5_kernel_size" : {"values" : [2, 3]},
        "mp_layer5_stride" : {"values" : [2, 3]},
        # batch norm conv
        "batch_norm_conv" : {"values" : [True, False]},
        "dropout_conv" : {"values" : [0, 0.1, 0.2]}, # if dont need use 0
        # Fully connected layers
        "no_of_fc_layers" : {"values" : [1]}, # Ignore the output layer
        "fc_activations" : {"values" : ["ReLU", "GELU", "SiLU", "SELU", "Mish", "LeakyReLU"]},
        "fc_neurons" : {"values" : [512, 1024]},
        "batch_norm_fc" : {"values" : [True]},
        "dropout_fc" : {"values" : [0, 0.2]}, 
        # Batch size
        "batch_size" : {"values" :  [16, 32, 64]},
        "data_aug" : {"values" : [True, False]},
        # Extras
        "num_workers" : {"values" : [10]},
        "epochs" : {"values" : [20]}
    }
}

In [10]:
sweep_id = "8km3fqmz" #wandb.sweep(sweep=sweep_config, project="Lightning_CNN", entity="A2_DA6401_DL")
wandb.agent(sweep_id, function=trainloop_sweep,project="Lightning_CNN", entity="A2_DA6401_DL")

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train_loss,█▇▆▆▆▇▅▄▄▃▃▃▃▃▂▂▂▂▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
val_acc,▁▃▃▁▂▁▅▅▅▄▆▆▇▆▇▅▆▇▇█
val_loss,▇▅▅█▇▇▃▄▅▅▃▃▂▂▂▃▃▁▂▁
epoch,19
train_loss,2.19639
trainer/global_step,2479
val_acc,0.226
val_loss,2.22264


wandb: Agent Starting Run: sr8lqrng with config:
wandb: 	batch_norm_conv: False
wandb: 	batch_norm_fc: True
wandb: 	batch_size: 32
wandb: 	conv_1_filter_size: 7
wandb: 	conv_2_filter_size: 5
wandb: 	conv_3_filter_size: 7
wandb: 	conv_4_filter_size: 7
wandb: 	conv_5_filter_size: 5
wandb: 	conv_activation: ReLU
wandb: 	conv_padding: [None, None, None, None, None]
wandb: 	conv_strides: [1, 1, 1, 1, 1]
wandb: 	data_aug: False
wandb: 	dropout_conv: 0.1
wandb: 	dropout_fc: 0
wandb: 	epochs: 20
wandb: 	fc_activations: SiLU
wandb: 	fc_neurons: 512
wandb: 	filter_mul: 0.5
wandb: 	input_channels: 3
wandb: 	input_size: [256, 256]
wandb: 	learning_rate: 0.001
wandb: 	mp_layer1_kernel_size: 2
wandb: 	mp_layer1_stride: 2
wandb: 	mp_layer2_kernel_size: 3
wandb: 	mp_layer2_stride: 3
wandb: 	mp_layer3_kernel_size: 5
wandb: 	mp_layer3_stride: 3
wandb: 	mp_layer4_kernel_size: 3
wandb: 	mp_layer4_stride: 3
wandb: 	mp_layer5_kernel_size: 2
wandb: 	mp_layer5_stride: 3
wandb: 	no_of_conv_blocks: 5
wandb: 	no

Not a good combination !!!


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train_loss,█▇▆▆▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
val_acc,▁▁▃▃▃▃▄▄▄▅▅▄▆▅▅▆▆█▇▇
val_loss,██▆▆▆▅▄▅▅▄▄▅▃▄▄▂▂▁▁▂
epoch,19
train_loss,2.17133
trainer/global_step,9979
val_acc,0.265
val_loss,2.17711


wandb: Sweep Agent: Waiting for job.
wandb: Ctrl + C detected. Stopping sweep.
